In [1]:
#standard libraries
import pandas as pd
import numpy as np

#visualization tools
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
sns.set_theme(style="whitegrid") 

#scikit-learn 
from sklearn.preprocessing import OrdinalEncoder,StandardScaler,OneHotEncoder
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor

from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

# xgboost

from xgboost import XGBRegressor

In [31]:
# datasetni yuklab df ga yuklab olaman
df = pd.read_csv('train_data.csv')
df

,id,airline,flight,source_city,departure_time,stops,arrival_time,destination_city,class,duration,days_left,price
0,1,Vistara,UK-810,Bangalore,Early_Morning,one,Night,Mumbai,Economy,14.25,21,7212
1,2,SpiceJet,SG-5094,Hyderabad,Evening,zero,Night,Kolkata,Economy,1.75,7,5292
2,3,Vistara,UK-846,Bangalore,Morning,one,Evening,Delhi,Business,9.58,5,60553
3,4,Vistara,UK-706,Kolkata,Morning,one,Evening,Hyderabad,Economy,6.75,28,5760
4,5,Indigo,6E-5394,Chennai,Early_Morning,zero,Morning,Mumbai,Economy,2.00,4,10712
...,...,...,...,...,...,...,...,...,...,...,...,...
19995,19996,Indigo,6E-6178,Bangalore,Night,one,Early_Morning,Mumbai,Economy,7.92,45,3153
19996,19997,AirAsia,I5-582,Kolkata,Morning,one,Afternoon,Delhi,Economy,5.83,24,3911
19997,19998,Vistara,UK-832,Chennai,Early_Morning,two_or_more,Evening,Bangalore,Economy,35.33,17,14822
19998,19999,Vistara,UK-996,Mumbai,Evening,one,Morning,Bangalore,Economy,16.33,21,6450


In [32]:
# create a column that is the km from source to destination
source = df['source_city'].value_counts().index
destination = df['destination_city'].value_counts().index
# Mumbai to 1408 Delhi, Mumbai to Bangalora 981, Mumbai to Kolkata 2051, Mumbai to Hyderabad 710, Mumbai to Chennai 1333
# Delhi to 1408 Mumbai, Delhi to Bangalora 1740, Delhi to Kolkata 1467, Delhi to Hyderabad 1255, Delhi to Chennai 2194
# Kolkata to 1467 Delhi, Kolkata to Bangalora 1861, Kolkata to Mumbai 2051, Kolkata to Hyderabad 1180, Kolkata to Chennai 1679
# Hyderabad to 1255 Delhi, Hyderabad to 1180 Kolkata, Hyderabad to Bangalore 574, Hyderabad to 710 Mumbai, Hyderabad to 2194 Chennai 634
# Chennai to 2194 Delhi, Chennai to 1679 Kolkata, Chennai to 1333 Mumbai, Chennai to Bangalore 345, Chennai to 634 Hyderabad
# Bangalore to 345 Hyderabad, Bangalore to Mumbai 981, Bangalore to Kolkata 1861, Bangalore to Delhi 1740, Bangalore to Chennai 345
km = []
for source, destination in zip(df['source_city'], df['destination_city']):
    if source == 'Mumbai':
        if destination == 'Delhi':
            km.append(1408)
        elif destination == 'Bangalore':
            km.append(981)
        elif destination == 'Kolkata':
            km.append(2051)
        elif destination == 'Hyderabad':
            km.append(710)
        elif destination == 'Chennai':
            km.append(1333)
    elif source == 'Delhi':
        if destination == 'Mumbai':
            km.append(1408)
        elif destination == 'Bangalore':
            km.append(1740)
        elif destination == 'Kolkata':
            km.append(1467)
        elif destination == 'Hyderabad':
            km.append(1255)
        elif destination == 'Chennai':
            km.append(2194)
    elif source == 'Kolkata':
        if destination == 'Mumbai':
            km.append(2051)
        elif destination == 'Bangalore':
            km.append(1861)
        elif destination == 'Delhi':
            km.append(1467)
        elif destination == 'Hyderabad':
            km.append(1180)
        elif destination == 'Chennai':
            km.append(1679)
    elif source == 'Hyderabad':
        if destination == 'Mumbai':
            km.append(710)
        elif destination == 'Delhi':
            km.append(1255)
        elif destination == 'Kolkata':
            km.append(1180)
        elif destination == 'Bangalore':
            km.append(574)
        elif destination == 'Chennai':
            km.append(634)
    elif source == 'Chennai':
        if destination == 'Mumbai':
            km.append(1333)
        elif destination == 'Bangalore':
            km.append(345)
        elif destination == 'Kolkata':
            km.append(1679)
        elif destination == 'Hyderabad':
            km.append(634)
        elif destination == 'Delhi':
            km.append(2194)
    elif source == 'Bangalore':
        if destination == 'Mumbai':
            km.append(981)
        elif destination == 'Delhi':
            km.append(1740)
        elif destination == 'Kolkata':
            km.append(1861)
        elif destination == 'Hyderabad':
            km.append(574)
        elif destination == 'Chennai':
            km.append(345)
df['km'] = km


In [52]:
from sklearn import model_selection
df['kfold'] = 1 # create a kfold column. agar bu columni qo'shmasak kfold bizga float qiymat qaytaradi.
kf = model_selection.KFold(n_splits=5,shuffle=True,random_state=99) # kfold function is used to split the data into 5 folds
for fold,(train_indicies,valid_indicies) in enumerate(kf.split(X=df)): # train_indicies and valid_indicies are the indices of the data. enumerate is used to get the index of the data
    df.loc[valid_indicies,'kfold'] = fold

In [53]:
df_test = pd.read_csv('test_data.csv')

# create a column that is the km from source to destination
source = df_test['source_city'].value_counts().index
destination = df_test['destination_city'].value_counts().index
# Mumbai to 1408 Delhi, Mumbai to Bangalora 981, Mumbai to Kolkata 2051, Mumbai to Hyderabad 710, Mumbai to Chennai 1333
# Delhi to 1408 Mumbai, Delhi to Bangalora 1740, Delhi to Kolkata 1467, Delhi to Hyderabad 1255, Delhi to Chennai 2194
# Kolkata to 1467 Delhi, Kolkata to Bangalora 1861, Kolkata to Mumbai 2051, Kolkata to Hyderabad 1180, Kolkata to Chennai 1679
# Hyderabad to 1255 Delhi, Hyderabad to 1180 Kolkata, Hyderabad to Bangalore 574, Hyderabad to 710 Mumbai, Hyderabad to 2194 Chennai 634
# Chennai to 2194 Delhi, Chennai to 1679 Kolkata, Chennai to 1333 Mumbai, Chennai to Bangalore 345, Chennai to 634 Hyderabad
# Bangalore to 345 Hyderabad, Bangalore to Mumbai 981, Bangalore to Kolkata 1861, Bangalore to Delhi 1740, Bangalore to Chennai 345
km = []
for source, destination in zip(df_test['source_city'], df_test['destination_city']):
    if source == 'Mumbai':
        if destination == 'Delhi':
            km.append(1408)
        elif destination == 'Bangalore':
            km.append(981)
        elif destination == 'Kolkata':
            km.append(2051)
        elif destination == 'Hyderabad':
            km.append(710)
        elif destination == 'Chennai':
            km.append(1333)
    elif source == 'Delhi':
        if destination == 'Mumbai':
            km.append(1408)
        elif destination == 'Bangalore':
            km.append(1740)
        elif destination == 'Kolkata':
            km.append(1467)
        elif destination == 'Hyderabad':
            km.append(1255)
        elif destination == 'Chennai':
            km.append(2194)
    elif source == 'Kolkata':
        if destination == 'Mumbai':
            km.append(2051)
        elif destination == 'Bangalore':
            km.append(1861)
        elif destination == 'Delhi':
            km.append(1467)
        elif destination == 'Hyderabad':
            km.append(1180)
        elif destination == 'Chennai':
            km.append(1679)
    elif source == 'Hyderabad':
        if destination == 'Mumbai':
            km.append(710)
        elif destination == 'Delhi':
            km.append(1255)
        elif destination == 'Kolkata':
            km.append(1180)
        elif destination == 'Bangalore':
            km.append(574)
        elif destination == 'Chennai':
            km.append(634)
    elif source == 'Chennai':
        if destination == 'Mumbai':
            km.append(1333)
        elif destination == 'Bangalore':
            km.append(345)
        elif destination == 'Kolkata':
            km.append(1679)
        elif destination == 'Hyderabad':
            km.append(634)
        elif destination == 'Delhi':
            km.append(2194)
    elif source == 'Bangalore':
        if destination == 'Mumbai':
            km.append(981)
        elif destination == 'Delhi':
            km.append(1740)
        elif destination == 'Kolkata':
            km.append(1861)
        elif destination == 'Hyderabad':
            km.append(574)
        elif destination == 'Chennai':
            km.append(345)
df_test['km'] = km


In [54]:
useful_features = [c for c in df.columns if c not in ("id", "price", "kfold")]

# categoriya ustunlarimiz
object_cols = ['airline','flight','source_city','departure_time','stops','arrival_time','destination_city','class','km'] 
df_test = df_test[useful_features]

In [15]:
final_predictions = []
for fold in range(5):
    xtrain =  df[df.kfold != fold].reset_index(drop=True)
    xvalid = df[df.kfold == fold].reset_index(drop=True)
    xtest = df_test.copy()

    ytrain = xtrain.price
    yvalid = xvalid.price
    
    xtrain = xtrain[useful_features]
    xvalid = xvalid[useful_features]
    
    ordinal_encoder = OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-999)
    xtrain[object_cols] = ordinal_encoder.fit_transform(xtrain[object_cols])
    xvalid[object_cols] = ordinal_encoder.transform(xvalid[object_cols])
    xtest[object_cols] = ordinal_encoder.transform(xtest[object_cols])
    
    model = XGBRegressor(random_state=fold, n_jobs=4)
    model.fit(xtrain, ytrain)
    preds_valid = model.predict(xvalid)
    test_preds = model.predict(xtest)
    final_predictions.append(test_preds)
    print(fold, mean_squared_error(yvalid, preds_valid, squared=False))

0 3551.2577077718443
1 3384.804069023423
2 3303.853903409122
3 3491.4336640959095
4 3561.6787938083994


In [58]:
final_predictions = []
for fold in range(5):
    xtrain =  df[df.kfold != fold].reset_index(drop=True)
    xvalid = df[df.kfold == fold].reset_index(drop=True)
    xtest = df_test.copy()

    ytrain = xtrain.price
    yvalid = xvalid.price
    
    xtrain = xtrain[useful_features]
    xvalid = xvalid[useful_features]
    
    ordinal_encoder = OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-999)
    xtrain[object_cols] = ordinal_encoder.fit_transform(xtrain[object_cols])
    xvalid[object_cols] = ordinal_encoder.transform(xvalid[object_cols])
    xtest[object_cols] = ordinal_encoder.transform(xtest[object_cols])
    
    model = RandomForestRegressor(random_state=fold, n_jobs=4, n_estimators=100)
    model.fit(xtrain, ytrain)
    preds_valid = model.predict(xvalid)
    test_preds = model.predict(xtest)
    final_predictions.append(test_preds)
    print(fold, mean_squared_error(yvalid, preds_valid, squared=False))

0 3521.280619159666
1 3239.1011543696686
2 3181.801541904555
3 3287.0316999729016
4 3527.2857692963985


In [59]:
baholash = np.mean(np.column_stack(final_predictions), axis=1)

In [60]:
sample_sub = pd.read_csv('sample_solution.csv')

In [61]:
sample_sub.price = baholash
sample_sub.to_csv("submission1.csv", index=False)
sample_sub

,id,price
0,1,55624.454
1,2,62238.956
2,3,23492.408
3,4,2396.184
4,5,5735.362
...,...,...
4995,4996,60252.792
4996,4997,4472.536
4997,4998,50257.274
4998,4999,49123.334
